<a href="https://colab.research.google.com/github/Artmalik92/Congruence-Test-For-Students/blob/main/geometry_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В системах геодезического мониторинга одним из методов обнаружения деформаций является построение ГНСС сетей и анализ движений в них при помощи вычисления невязок в замкнутых фигурах или геометрической модели. Геометрическая модель использует геометрические свойства объектов. Например, можно использовать геометрические связи между точками (углы и расстояния) для определения их относительного положения и оценки стабильности измерений. Данный принцип можно реализовать в виде программы геометрического теста на Python.

Геометрический тест стабильности позволяет выявлять аномальные движения в замкнутой геодезической сети путем вычисления разностей параметров этой сети (внутренних углов, длин базовых линий или координат) на начальную и i-ую эпоху. После этого производится статистическая оценка разностей. Если сумма разностей между параметрами статистически не значима, то эти две фигуры конгруэнтны, т. е. аномальных движений нет.

Библиотеки, которые нам понадобятся:

In [ ]:
import pandas
import numpy
from scipy.stats import chi2

Библиотека **pandas** необходима для обработки и структурирования данных в виде таблиц.
**Numpy** используется для выполнения математических операций с матрицами и другими массивами. **Scipy** содержит модуль **stats**, который позволяет выполнять в Python статистические операции, в частности тест Хи-квадрат, который мы будем использовать.

In [ ]:

data = pandas.read_csv('generated_coordinates_noise002_offset002.csv')
print(data)

FileNotFoundError: [Errno 2] No such file or directory: 'generated_coordinates_noise002_offset002.csv'